In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
#Import packages
#import gym
#import gym_uav
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from itertools import count
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [33]:
#Import Custom Classes

from Source.NN_model_2 import QNetwork
from Source.dqn_agent_2 import Agent, ReplayBuffer, EpsilonGreedyStrategy
from Source.Env_Manager import EnvManager
from Source.Misc import plot



In [34]:
#Hyper-parameters
BUFFER_SIZE = int(1e5)      #replay buffer size
BATCH_SIZE = 60             #minibatch size
GAMMA = 0.999                #discount factor
TAU = 1e-3                  #for soft update of target parameters
LR = 5e-4                   #learning rate
UPDATE_EVERY = 50            #how often to update the network
eps_start = 1
eps_end = 0.01
eps_decay = 0.9985 #125e-6
train_episodes = 3200
test_episodes = 1
seed = 0                    #random seed number
episode_step_limit = 50

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Choose the environment
em = EnvManager(device, 'uav-v2', seed)
available_actions = em.num_actions_available()
random.seed(seed)


#Select the strategy
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)

#Initialize the agent
agent = Agent(strategy, available_actions, seed, device)

#Instantiate MemoryBuffer
memory = ReplayBuffer(available_actions, BUFFER_SIZE, BATCH_SIZE, seed, device)

policy_net = QNetwork(available_actions, seed).to(device)
target_net = QNetwork(available_actions, seed).to(device)
print(policy_net)

#Initialize target_net weights to policy_net weights
target_net.load_state_dict(policy_net.state_dict())
target_net.eval() #Set the target_net in eval mode

#Select the optimizer
optimizer = optim.Adam(params=policy_net.parameters(), lr=LR)

QNetwork(
  (fc1): Linear(in_features=2, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=32, bias=True)
)


In [36]:
rate_tr = 0.0
t_step = 0
ue_xloc = em.env.ue_xloc  #10 locs
ue_yloc = em.env.ue_yloc[::-1]     #5 locs

rate_arr = np.zeros([(max(ue_yloc)-min(ue_yloc))//50 +1, (max(ue_xloc)-min(ue_xloc))//50 + 1]) #(-500,50) -----> (500,500)
for y_ndx in range(len(ue_yloc)):
    for x_ndx in range(len(ue_xloc)):
        state = np.array([ue_xloc[x_ndx], ue_yloc[y_ndx]])/ em.env.high_obs
        exh_bdir,exh_rate = em.env.get_Exh_Rate(state)
        rate_arr[y_ndx,x_ndx] = np.around(exh_rate, decimals=3)

row_labels = [str(x) for x in ue_yloc]
col_labels = [str(x) for x in ue_xloc]

fig, ax = plt.subplots(1,1, figsize=(10,10))
im = ax.imshow(rate_arr, aspect='auto')

# We want to show all ticks...
ax.set_xticks(np.arange(rate_arr.shape[1]+1)-.5)
ax.set_yticks(np.arange(rate_arr.shape[0]+1)-.5)
# ... and label them with the respective list entries
ax.set_xticklabels(col_labels, fontsize=8)
ax.set_yticklabels(row_labels, fontsize=8)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="center",
         rotation_mode="anchor")
plt.setp(ax.get_yticklabels(), rotation=0, ha="center",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(rate_arr.shape[0]):
    for j in range(rate_arr.shape[1]):
        text = ax.text(j, i, rate_arr[i, j],
                       ha="center", va="center", color="w", fontsize=8)

ax.set_title("Max possible Rates of UAV (w.r.t. BS)")
fig.tight_layout()
plt.show()

KeyboardInterrupt: 

In [ ]:
rate_arr = rate_arr.flatten()

plt.plot(rate_arr)
plt.show()

In [ ]:
ang=np.arange(np.pi/8, 10*np.pi/8, np.pi/8)
print(ang)
tan_ang=[]
for a in ang:
    tan_ang.append(np.arctan2(np.sin(a), np.cos(a)))
    
print(tan_ang)

Compute and Set Rate Threshold

In [ ]:
print(rate_arr[100])
rate_thr = rate_arr[100]

Testing Environment with untrained Agent

In [ ]:
state = em.reset(rate_thr)
#state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
print(state.shape)
for j in range(5):
    action = agent.act(state, policy_net)
    print("action: ", action.item())
    #print("Action: {}, Qval: {}".format(action, qval))
    #env.render()
    #print("[NB] action: {}".format(action))
    state, reward, done, _ = em.step(action)
    #state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
    if done:
        break 
        
em.env.render()      
em.close()
print("UAV Source: {0}, UAV dest: {1}".format((em.env.ue_xsrc, em.env.ue_ysrc), (em.env.ue_xdest[0], em.env.ue_ydest[0]) ))

In [ ]:
'''
nrows=len(ue_yloc)+1
ncols=len(ue_xloc)+1


xsrc_ndx = em.env.ue_xsrc

map_str=""
row_str = ""
for c_ndx in range(1, ncols):
    if c_ndx == ncols:
        row_str +="--\n"
    else:
        row_str += "--" 
map_str += row_str +"\n"
    
for r_ndx in range(1,nrows):
    row_str = ""
    for c_ndx in range(1, ncols):
        if c_ndx == ncols:
            row_str +="|\n"
        else:
            row_str += "|  " #+ "(" + str(r_ndx) + "," + str(c_ndx) + ")" 
    map_str += row_str +"\n"
    
    row_str = ""
    for c_ndx in range(1, ncols):
        if c_ndx == ncols:
            row_str +="--\n"
        else:
            row_str += "--" 
    map_str += row_str +"\n"
    
print(map_str)
'''    
    

Training the DQN Agent

In [ ]:
#Training Loop
episode_durations = []
episode_rewards = []
policy_net.train()

for episode in range(train_episodes):
    state = em.reset(rate_thr)
    
    #state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
    ep_loss = 0.0
    ep_rwd = 0.0
    timestep = 0
    agent.current_step +=1
    for timestep in count():
        action = agent.act(state, policy_net)
        next_state, reward, done, _ = em.step(action)
        ep_rwd += reward.item()
        memory.add(state, action, reward, next_state, done)
        
        state = next_state
        #state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)
        
        if memory.can_provide_sample():
            experiences = memory.sample()
            states, actions, rewards, next_states, dones = experiences
            
            #print(states.shape, states.dtype)
            #print(actions.unsqueeze(-1).shape)
            current_q_values = policy_net(states).gather(1,index=actions.unsqueeze(-1))
            next_q_values = target_net(next_states).detach().max(1)[0]
            target_q_values = (next_q_values*GAMMA) + rewards
            
            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
            #print("loss: ", loss.item())
            ep_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        if (timestep==episode_step_limit) or done:
            episode_durations.append(timestep)
            episode_rewards.append(ep_rwd)
            print('\rEpisode {}\t timestep: {},\tScore: {:.2f}, eps: {}'.format(episode, timestep, ep_rwd, agent.strategy.get_exploration_rate(agent.current_step)))
            plot(episode_rewards, 100)
            break
            
        
    if episode % UPDATE_EVERY == 0:
        target_net.load_state_dict(policy_net.state_dict())
    
    if (np.mean(episode_rewards[-100:]) >= 20000):
        print("Goal is reached in {} episodes!\n".format(episode))
        break
        
    torch.save(policy_net.state_dict(), 'checkpoint.pth')
    #em.close()

Testing the DQN agent

In [ ]:
#Load the weights from file
policy_net.load_state_dict(torch.load('checkpoint.pth'))

rate_acc_los_scores = []
rate_acc_exh_scores = []
learnt_val_fns = []
true_val_fns = []
test_scores = []
policy_net.eval()
for i_episode in range(1):
    
    learnt_rates = 0.0
    los_rates = 0.0
    exh_rates = 0.0
    state_tensor = em.reset(rate_thr)
    learnt_val_fn = 0.0
    true_val_fn = 0.0
    perf_score = 0
    for t_step in count():
        action = policy_net(state_tensor).argmax(dim=1).to(device)#agent.act(state_tensor, policy_net)
        next_state_tensor, reward, done, _ = em.step(action)
        
        
        state = np.rint(state_tensor.squeeze().cpu().data.numpy() * em.env.high_obs)
        next_state = np.rint(next_state_tensor.squeeze().cpu().data.numpy() * em.env.high_obs)
        print("Current State: ", state)
        print("Next State: ", next_state)
        
        curr_loc = (state[0], state[1])
        next_loc = (next_state[0], next_state[1])
        #print("Test Episode: {2}, Current Location: {0}, Next Location: {1}".format(curr_loc, next_loc, i_episode))
        #print("Learnt Action: ", test_net(state))
        
            
        next_state = next_state / em.env.high_obs
        state = state / em.env.high_obs
        
        state_tensor = torch.tensor(np.array([next_state]), dtype=torch.float32).to(device)
        
        
        
        #rate measurements
        learnt_rate = em.env.get_Rate()
        _,los_rate = em.env.get_Los_Rate(state)
        exh_bdir,exh_rate = em.env.get_Exh_Rate(state)
        
        if (learnt_rate > em.env.rate_threshold):
            perf_score +=1
        learnt_val_fn += ((GAMMA**(t_step)) * (learnt_rate))
        true_val_fn += ((GAMMA**(t_step)) * (exh_rate))
        
        rbd_ndx, ue_mv_ndx = em.env.decode_action(action)
        ue_vx, ue_vy = em.env.choose_vel(ue_mv_ndx)
        print("time step: ", t_step+1)
        print("action parameters: ue_vx: {}, ue_vy: {}".format(ue_vx, ue_vy))
        print("exh dir: {0}, learnt bdir: {1}, learnt (AoA-AoD): {2}".format(exh_bdir, em.env.BeamSet[rbd_ndx], em.env.aoa-em.env.aod))
        print("Rwd: {3}, Learnt Rate: {0}, Exh_Rate: {1}, Los_Rate: {2}\n\n".format(learnt_rate, exh_rate, los_rate, reward))
        
       
        if (t_step== episode_step_limit) or done:
            em.env.render()
            break
        '''
        
        #Move to next_state
        state = next_state
        
        learnt_rates += learnt_rate
        los_rates += los_rate
        exh_rates += exh_rate
        
        if done:
            rate_acc_los = learnt_rates/ los_rates
            rate_acc_exh = learnt_rates / exh_rates
            rate_acc_los_scores.append(rate_acc_los)
            rate_acc_exh_scores.append(rate_acc_exh)
            learnt_val_fns.append(learnt_val_fn)
            true_val_fns.append(true_val_fn)
            test_scores.append(perf_score)
            #writer.add_scalar("test/rate_acc_los", rate_acc_los, i_episode)
            #writer.add_scalar("test/rate_acc_exh", rate_acc_exh, i_episode)
            print('\rEpisode {}\tRate_acc_los Score: {:.2f}\tRate_acc_exh Score: {:.2f}\t perf_score: {:.2f}'.format(i_episode, rate_acc_los, rate_acc_exh, perf_score, end=""))
            print("\n\n")
            break
        '''
#print("Total Epsiodes {},Average Rate_Acc_los: {:.2f}, Average Rate_Acc_exh: {:.2f}\n".format(test_episodes, np.mean(rate_acc_los_scores), np.mean(rate_acc_exh_scores)))

Compute the value function

In [ ]:
#Load the weights from file
policy_net.load_state_dict(torch.load('checkpoint.pth'))
test_episodes = 20

rate_acc_los_scores = []
rate_acc_exh_scores = []
perf_scores = []
learnt_val_fns = []
true_val_fns = []
test_scores = []
policy_net.eval()
for i_episode in range(1, test_episodes+1):
    
    learnt_rates = 0.0
    los_rates = 0.0
    exh_rates = 0.0
    state_tensor = em.reset(rate_thr)
    learnt_val_fn = 0.0
    true_val_fn = 0.0
    perf_score = 0
    
    for t_step in count():
        action = policy_net(state_tensor).argmax(dim=1).to(device)#agent.act(state_tensor, policy_net)
        
        next_state_tensor, reward, done, _ = em.step(action)
        
        state = np.rint(state_tensor.squeeze().cpu().data.numpy() * em.env.high_obs)
        next_state = np.rint(next_state_tensor.squeeze().cpu().data.numpy() * em.env.high_obs)
        #print("Current State: ", state)
        #print("Next State: ", next_state)

        curr_loc = (state[0], state[1])
        next_loc = (next_state[0], next_state[1])
        #print("Test Episode: {2}, Current Location: {0}, Next Location: {1}".format(curr_loc, next_loc, i_episode))
        #print("Learnt Action: ", test_net(state))
        
            
        next_state = next_state / em.env.high_obs
        
        state_tensor = torch.tensor(np.array([next_state]), dtype=torch.float32).to(device)
        
        #rate measurements
        learnt_rate = em.env.get_Rate()
        _,los_rate = em.env.get_Los_Rate(next_state)
        exh_bdir,exh_rate = em.env.get_Exh_Rate(next_state)
        
        learnt_val_fn += ((GAMMA**(t_step)) * (learnt_rate))
        true_val_fn += ((GAMMA**(t_step)) * (exh_rate))
        
        #rbd_ndx, ue_vx_ndx, ue_vy_ndx = em.env.decode_action(action)
        #print("time step: ", t_step+1)
        #print("action parameters: ue_vx: {0}, ue_vy: {1}".format(em.env.ue_vx[ue_vx_ndx], em.env.ue_vx[ue_vx_ndx]))
        #print("exh dir: {0}, learnt bdir: {1}".format(exh_bdir, em.env.BeamSet[rbd_ndx]))
        #print("Rwd: {3}, Learnt Rate: {0}, Exh_Rate: {1}, Los_Rate: {2}\n\n".format(learnt_rate, exh_rate, los_rate, reward))
    
        if (learnt_rate >= em.env.rate_threshold):
            perf_score +=1

        los_rates+=los_rate
        exh_rates+=exh_rate
        learnt_rates+=learnt_rate
        
        if (t_step== episode_step_limit) or done:
            rate_acc_los = learnt_rates/ los_rates
            rate_acc_exh = learnt_rates / exh_rates
            print("\rTest epsiode: {}\t rate_acc_los: {:.2f}\trate_acc_exh: {:.2f}, episode_len: {}, perf_score: {}".format(i_episode, rate_acc_los, rate_acc_exh, t_step, perf_score))
            rate_acc_los_scores.append(rate_acc_los)
            rate_acc_exh_scores.append(rate_acc_exh)
            #perf_scores.append(perf_score / t_step)
            learnt_val_fns.append(learnt_val_fn)
            true_val_fns.append(true_val_fn)
            break

In [ ]:
#print(learnt_val_fns)
#print(true_val_fns)
print("Avg rate_los_acc: {}, Avg rate_exh_acc: {}".format(np.mean(rate_acc_los_scores), np.mean(rate_acc_exh_scores)))



# plot the histogram
w=0.3
fig = plt.figure(figsize=(10,10))
x_axis = np.arange(test_episodes)
ax = fig.add_subplot(111)
rects1 = ax.bar(x_axis-w, learnt_val_fns, w, color='b')
rects2= ax.bar(x_axis, true_val_fns, w, color='g')
#plt.legend(loc='upper right')
ax.legend( (rects1[0], rects2[0]), ('rl_val_fn', 'exh_val_fn'), loc='upper right' )
plt.ylabel('Value Function')
plt.xlabel('Episode #')
ax.set_xticks(x_axis)
ax.set_yticks(np.arange(0.0,100.0,10))

def autolabel(rects):
    for rect in rects:
        h = rect.get_height()
        #ax.text(rect.get_x()+rect.get_width()/2., 1.03*h, '%.2f'%np.around(h, decimals=2),
        #        ha='center', va='bottom')

ax.axhline(np.mean(learnt_val_fns), color='b', linewidth=0.75, linestyle='--')
ax.axhline(np.mean(true_val_fns), color='g', linewidth=0.75, linestyle='--')
autolabel(rects1)
autolabel(rects2)
plt.show()